In [2]:
model_name = 'conformer'

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam

from sklearn.model_selection import train_test_split

import numpy as np

from braindecode.models import EEGConformer

from tensorflow.keras import utils as np_utils
from torch.utils.data import DataLoader, TensorDataset, random_split

C:\Users\rosit\AppData\Local\Programs\Python\Python310\lib\site-packages\moabb\pipelines\__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(


In [4]:
from processing_eeg_methods.data_utils import (
    convert_into_independent_channels,
    get_dataset_basic_info,
    get_input_data_path,
    standard_saving_path,
    write_model_info,
)
from processing_eeg_methods.data_loaders import load_data_labels_based_on_dataset
from processing_eeg_methods.share import datasets_basic_infos

subject_id = 8  # Only two things I should be able to change
dataset_name = "braincommand"  # Only two things I should be able to change

get_dataset_basic_info(datasets_basic_infos, dataset_name)
dataset_info: dict = datasets_basic_infos[dataset_name]

data_path = r"C:\Users\rosit\Documents\workprojects\bci_complete\EEG-Classifiers-Ensemble\Datasets\braincommand_dataset"

In [4]:
epochs_calibration, X_calibration, y_calibration_original = load_data_labels_based_on_dataset(
    dataset_info,
    subject_id,
    data_path,
    game_mode="calibration3",
)

epochs_singleplayer, X_singleplayer, y_singleplayer_original = load_data_labels_based_on_dataset(
    dataset_info,
    subject_id,
    data_path,
    game_mode="singleplayer",
)
y_calibration = [0] * len(y_calibration_original)
y_singleplayer = [1] * len(y_singleplayer_original)

X = np.concatenate((X_calibration, X_singleplayer), axis=0)
y = y_calibration + y_singleplayer

label 0 is 85
label 1 is 102
label 2 is 65
label 3 is 23
Not setting metadata
275 matching events found
Setting baseline interval to [0.0, 1.296] s
Applying baseline correction (mode: mean)
0 projection items activated


In [32]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

filepath = r"C:\Users\rosit\Documents\MATLAB\clean EEG BrainCommand\datasets_edited/8_calibration3_cleaned.set"
epochs = mne.io.read_epochs_eeglab(filepath, verbose=True)
X = epochs.get_data()
y = epochs.events[:, 2].astype(np.int64)
y = y - 1
dataset_info["#_channels"] = 6

Extracting parameters from C:\Users\rosit\Documents\MATLAB\clean EEG BrainCommand\datasets_edited\8_calibration3_cleaned.set...
Not setting metadata
166 matching events found
No baseline correction applied
0 projection items activated
Ready.


In [33]:
# Split the data into 50% training and 50% temp (which will later be split into test and validation)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.5, random_state=42)

# Split the temp data into 50% test and 50% validation, resulting in 25% of the original data each
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
print(y_train)
num_classess = len(set(y_train))
print(f'There are {num_classess} unique classes in the dataset')

[1 2 2 3 2 1 0 0 3 2 1 1 1 3 3 0 1 1 3 2 1 1 1 0 2 0 3 3 3 3 1 0 3 1 0 3 3
 0 2 1 2 3 3 3 3 2 2 2 2 3 3 0 1 2 1 1 0 0 2 0 2 2 0 0 3 1 2 0 0 0 1 1 1 0
 1 1 3 3 1 0 1 1 3]
There are 4 unique classes in the dataset


In [34]:
y_train

array([1, 2, 2, 3, 2, 1, 0, 0, 3, 2, 1, 1, 1, 3, 3, 0, 1, 1, 3, 2, 1, 1,
       1, 0, 2, 0, 3, 3, 3, 3, 1, 0, 3, 1, 0, 3, 3, 0, 2, 1, 2, 3, 3, 3,
       3, 2, 2, 2, 2, 3, 3, 0, 1, 2, 1, 1, 0, 0, 2, 0, 2, 2, 0, 0, 3, 1,
       2, 0, 0, 0, 1, 1, 1, 0, 1, 1, 3, 3, 1, 0, 1, 1, 3], dtype=int64)

In [35]:
kernels, chans, samples = 1, dataset_info["#_channels"], dataset_info["samples"]

# y_train = y_train - 1
# y_val = y_val - 1
# y_test = y_test - 1

# X_train      = X_train.reshape(X_train.shape[0], chans, samples)
# X_val   = X_val.reshape(X_val.shape[0], chans, samples)
# X_test       = X_test.reshape(X_test.shape[0], chans, samples)

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)  # Keep as integers

X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create datasets and loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

X_train shape: (83, 7, 325)
83 train samples
41 test samples


In [36]:
# Calculate the number of batches in train_loader
train_loader_size = len(train_loader)
print(f"Number of batches in train_loader: {train_loader_size}")

Number of batches in train_loader: 2


In [37]:
from collections import Counter

counts = Counter(y)

total_samples = sum(counts.values())

num_classes = len(counts)

class_weights = {class_label-1: total_samples / (num_classes * count) for class_label, count in counts.items()}

print(class_weights)

{-1: 0.9651162790697675, 0: 0.9431818181818182, 1: 1.0921052631578947, 2: 1.0121951219512195}


In [38]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = 'cpu'

In [39]:
n_classes = dataset_info["#_class"]
classes = list(range(n_classes))

n_outputs = dataset_info["#_class"]
n_chans = dataset_info["#_channels"]
n_filters_time = 40
filter_time_length = 25
pool_time_length = 75
pool_time_stride = 15
drop_prob = 0.5
att_depth = 6
att_heads = 10
att_drop_prob = 0.5
final_fc_length = 640
return_features = False
n_times = dataset_info["samples"]
chs_info = None
input_window_seconds = None
sfreq = dataset_info["sample_rate"]
add_log_softmax = True

# Initialize the EEGConformer model
conformer = EEGConformer(
    n_outputs=n_outputs,
    n_chans=n_chans,
    n_filters_time=n_filters_time,
    filter_time_length=filter_time_length,
    pool_time_length=pool_time_length,
    pool_time_stride=pool_time_stride,
    drop_prob=drop_prob,
    att_depth=att_depth,
    att_heads=att_heads,
    att_drop_prob=att_drop_prob,
    final_fc_length=final_fc_length,
    return_features=return_features,
    n_times=n_times,
    chs_info=chs_info,
    input_window_seconds=input_window_seconds,
    sfreq=sfreq,
    add_log_softmax=add_log_softmax
)
conformer = conformer.to(device)

print(conformer)

C:\Users\rosit\AppData\Local\Programs\Python\Python310\lib\site-packages\braindecode\models\base.py:180: UserWarning: LogSoftmax final layer will be removed! Please adjust your loss function accordingly (e.g. CrossEntropyLoss)!
  warnings.warn("LogSoftmax final layer will be removed! " +


Layer (type (var_name):depth-idx)                            Input Shape               Output Shape              Param #                   Kernel Shape
EEGConformer (EEGConformer)                                  [1, 6, 325]               [1, 4]                    --                        --
├─_PatchEmbedding (patch_embedding): 1-1                     [1, 1, 6, 325]            [1, 16, 40]               --                        --
│    └─Sequential (shallownet): 2-1                          [1, 1, 6, 325]            [1, 40, 1, 16]            --                        --
│    │    └─Conv2d (0): 3-1                                  [1, 1, 6, 325]            [1, 40, 6, 301]           1,040                     [1, 25]
│    │    └─Conv2d (1): 3-2                                  [1, 40, 6, 301]           [1, 40, 1, 301]           9,640                     [6, 1]
│    │    └─BatchNorm2d (2): 3-3                             [1, 40, 1, 301]           [1, 40, 1, 301]           80              

Do this to find the right final_fc_length

In [40]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(conformer.parameters(), lr =  0.0002,betas  = [0.5, 0.999])

In [41]:
import torch

conformer.to(device)

# Create a dummy input with the correct shape and move it to the same device
dummy_input = torch.randn(kernels, chans, samples).to(device)  # Batch size

# Pass the dummy input through the model up to the transformer encoder
try:
    with torch.no_grad():  # Disable gradient calculation for inference
        x = torch.unsqueeze(dummy_input, dim=1)  # Add an extra dimension to match input shape
        x = conformer.patch_embedding(x)  # Pass through Patch Embedding
        x = conformer.transformer(x)  # Pass through Transformer Encoder
        
        # Get the shape after the transformer and calculate the new `final_fc_length`
        print(f"Output shape after transformer: {x.shape}")
        final_fc_length_calculated = x.shape[1] * x.shape[2]
        print(f"Calculated `final_fc_length`: {final_fc_length_calculated}")

except Exception as e:
    print("Error during partial forward pass:", str(e))

Output shape after transformer: torch.Size([1, 16, 40])
Calculated `final_fc_length`: 640


In [42]:
num_epochs = 250
## TRAIN
for epoch in range(num_epochs):
    conformer.train()
    
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()
        
        outputs = conformer(X_batch)
        loss = criterion(outputs, y_batch)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == y_batch).sum().item()
        total_predictions += y_batch.size(0)
    
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = correct_predictions / total_predictions

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

## Validation
    conformer.eval()
    val_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            outputs = conformer(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == y_batch).sum().item()
            total_predictions += y_batch.size(0)
    
    val_loss /= len(val_loader)
    val_accuracy = correct_predictions / total_predictions

    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")


EinopsError: Shape mismatch, 2 != 1

In [ ]:
## TEST
test_loss = 0.0
correct_predictions = 0
total_predictions = 0

criterion = torch.nn.CrossEntropyLoss()

with torch.no_grad():
    for X_batch, y_batch in test_loader:

        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        outputs = conformer(X_batch)
        
        loss = criterion(outputs, y_batch)
        test_loss += loss.item()

        _, predicted = torch.max(outputs, 1)  # Get the class with the highest score
        correct_predictions += (predicted == y_batch).sum().item()
        total_predictions += y_batch.size(0)

test_loss /= len(test_loader)
test_accuracy = correct_predictions / total_predictions

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")